**Часть 1**

**C mallet у меня ошибка, как у всех, поэтому я брала gensim**

In [162]:
#!pip install pyLDAvis
#!pip install nltk
#!pip install spacy

In [163]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import nltk; nltk.download('stopwords')


!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [164]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [165]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [168]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- ']


In [169]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [170]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [171]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [172]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [173]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [174]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

**Часть 2**

In [175]:
def which_is_better(corpus):  
  coh_score = []  # список, где хранятся значения coherence score
  n = range(10,16)  # для итерации по числу топиков 
  num_top = list(n)  # список с числом топиков (нужно для словаря)

# обучаем модель
  for i in n:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= int(i), 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
  
# находим coherence_score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    coh_score.append(coherence_lda)

# создаем словарь coherence score : число топиков 
  dictionary = dict(zip(coh_score, num_top))

  a = max(coh_score)  # найдем максимум coh_score 

  best = dictionary[a]  

  return best

In [176]:
best = which_is_better(corpus)

**Часть 3**

In [177]:
print(int(best))

10


In [178]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= int(best), 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [179]:
spisok = lda_model.print_topics()
doc_lda = lda_model[corpus]


Разделим слова и веса слов на два списка

In [180]:
s = 0
slov = []  # список списков слов
ves = []  # список списков весов
for i in spisok:
  slova = ''
  vesa = ''
  a = (spisok[s][1])
  s+=1
  slova = re.sub(r'[^\w\s]+|[\d]+', r'',a).strip()  # убираем лишнее
  vesa = re.sub(r'[A-Za-z"*+_]', r'',a).strip()  # убираем лишнее
  ves.append((vesa).split(' ')[::2])
  slov.append((slova).split(' ')[::2])

Создадим словарь, чтобы иметь доступ к весу по слову

In [181]:
import itertools

In [182]:
merged_vesa = list(itertools.chain.from_iterable(ves))  # список весов
merged_vesa[::40]

['0.028', '0.090', '0.021']

In [183]:
merged_slova = list(itertools.chain.from_iterable(slov))  # список слов
merged_slova[::90]

['team', 'key']

In [184]:
d = dict(zip(merged_slova, merged_vesa))


In [185]:
sf = 0
text_numb = []
most_imp = []

for lemma in data_lemmatized:
  text_numb.append(sf)
  sf += 1
  # нужно 10 весов, тк лучший coherent score с 10 топиками
  null, one, two, three, four, five = 0, 0, 0, 0, 0, 0
  six, seven, eight, nine = 0, 0, 0, 0

  for l in lemma:
    if l in slov[0]:
      null += float(d[l])
    elif l in slov[1]:
      one += float(d[l])
    elif l in slov[2]:
      two += float(d[l])
    elif l in slov[3]:
      three += float(d[l])
    elif l in slov[4]:
      four += float(d[l])
    elif l in slov[5]:
      five += float(d[l])
    elif l in slov[6]:
      six += float(d[l])
    elif l in slov[7]:
      seven += float(d[l])
    elif l in slov[8]:
      eight += float(d[l])
    elif l in slov[9]:
      nine += float(d[l])
    else:
      pass
  topics = ['null', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
  vesy = (str(null) + ' ' + str(one) + ' ' + str(two) + ' ' + str(three) + ' ' + str(four) + 
        ' ' + str(five) + ' ' + str(six) + ' ' + str(seven) + ' ' + str(eight) + ' ' + str(nine)).split(' ')
  dic = dict(zip(vesy, topics))
  a = str(max (null, one, two, three, four, five, six, seven, eight, nine))
  most_imp.append(dic[a])
print(most_imp[::100])  #  главный топик для каждого текста

['three', 'six', 'seven', 'seven', 'six', 'seven', 'six', 'nine', 'six', 'seven', 'one', 'seven', 'one', 'null', 'one', 'seven', 'one', 'six', 'seven', 'seven', 'seven', 'six', 'one', 'nine', 'six', 'eight', 'seven', 'six', 'two', 'seven', 'seven', 'five', 'one', 'three', 'eight', 'one', 'six', 'null', 'seven', 'seven', 'nine', 'six', 'seven', 'seven', 'seven', 'seven', 'three', 'six', 'seven', 'seven', 'seven', 'six', 'eight', 'one', 'seven', 'seven', 'six', 'seven', 'eight', 'seven', 'one', 'six', 'six', 'six', 'seven', 'seven', 'six', 'seven', 'one', 'six', 'seven', 'eight', 'seven', 'six', 'six', 'one', 'seven', 'null', 'seven', 'five', 'seven', 'seven', 'seven', 'six', 'six', 'six', 'nine', 'five', 'null', 'seven', 'seven', 'seven', 'six', 'six', 'seven', 'six', 'seven', 'six', 'seven', 'two', 'seven', 'seven', 'seven', 'three', 'six', 'seven', 'seven', 'one', 'five', 'seven', 'seven', 'seven', 'seven', 'eight']


In [187]:
# объединим тексты с одинаковым главным топиком
s = 0
sp0, sp1, sp2, sp3, sp4, sp5, sp6, sp7, sp8, sp9 = [], [], [], [], [], [], [], [], [], []
for i in most_imp:
  if i == 'null':
    sp0.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'one':
    sp1.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'two':
    sp2.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'three':
    sp3.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'four':
    sp4.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'five':
    sp5.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'six':
    sp6.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'seven':
    sp7.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'eight':
    sp8.append(' '.join(data_lemmatized[s]))
    s += 1
  elif i == 'nine':
    sp9.append(' '.join(data_lemmatized[s]))
    s += 1
  else:
    s += 1

**Часть 4**

In [188]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tf_idf_vectorize = TfidfVectorizer(use_idf=True)

Получим датафрейм; столбец text - текст, topic_number - номер топика ( всего их у нас 10, те от 0 до 9 включительно), five_words - пять самых важных слов, определенных при помощи TfidfVectorizer

In [189]:
# функция итерирует по топикам с текстами, выбирая для каждого текста в каждом топика 5 главных слов

df = pd.DataFrame(columns=['text', 'topic_number', 'five_words'])  # итоговый датафрейм
texts = [sp0, sp1, sp2, sp3, sp4, sp5, sp6, sp7, sp8, sp9]  # тексты по топикам
top = 0

for t in texts:  # итерация по топикам
  s = 0
  vectors = tf_idf_vectorize.fit_transform(t)

  for i in t:  # итерация по текстам внутри топика
    vec = vectors[s]
    dfvec = pd.DataFrame(vec.T.todense(), index=tf_idf_vectorize.get_feature_names(), columns=["tf_idf"]) 
    five_words = list(dfvec.sort_values(by=["tf_idf"],ascending=False).index[:5])
    df = df.append({'text': i, 'topic_number': top, 'five_words': ', '.join(five_words)}, ignore_index=True)
    s+=1

  top+=1

df[::100]

,text,topic_number,five_words
0,win icon help please line win download several...,0,"icon, help, bmp, wallpaper, appreciated"
100,obispo line strike else silly impose game susp...,0,"suspension, sad, impose, mathematic, whittier"
200,sad write foul illegal contact player stick bo...,0,"foul, penalty, penalty_shot, award, shot"
300,reply organization line article session write ...,0,"israeli, attack, session, target, fair"
400,medium wing epa host epa utoronto_ca line news...,0,"epa, tune, leafs, wish, wing"
...,...,...,...
10900,write canonical book add much late churchs his...,8,"whoah, add, book, protestant, much"
11000,window keeps_crashing help line nntp_poste hos...,9,"stack, crash, keeps_crashing, set, apr_gmt"
11100,help change line write crosspost vga card corr...,9,"specially, file, bmp, compress, screen"
11200,net security xxdate_fri line call paper intern...,9,"submission, symposium, chair, panel, security"


**Часть 5**

**Подсчет Coherence score**

1 этап - набор слов t (то есть, по сути, набор данных) делится на набор пар подмножеств слов S

2 этап - вероятности слов P вычисляются на основе заданного эталонного корпуса

3 этап - S и P нужны для подсчета согласованности между парами подмножеств S

4 этап - объединение значений до одного значения когерентности c

Для Topic coherence используется две метрики для подсчета согласованности темы - внутренняя и внешняя

Внутренняя - представлена как Umass. Проводит измерения, чтобы сравнить слово только с предыдущими и последующими словами соответственно, поэтому нужен упорядоченный набор слов.  Используется как функция попарной оценки в виде эмпирической условной логарифмической вероятности со сглаживающим подсчетом, чтобы избежать вычисления логарифма нуля

Внешняя - представлена как UCI. В измерении UCI каждое отдельное слово сопряжено с каждым другим отдельным словом. UCIcoherence использует PMI. PMI - рассчитывается как log  от деления числа документов (содержащих оба слова) на произведения количества документов (содержащих слово 1) и количества документов (содержащих слово 2)


Как внутренняя, так и внешняя мера вычисляют балл когерентности c (сумма попарных баллов по словам w1,..., wn, используемым для описания темы)